# 한빛아카데미 - 새로 나온 책 10페이지 
- https://www.hanbit.co.kr/academy/books/new_book_list.html
- 제목, 저자, 출간일, 쪽수, 가격, 역자

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
url = base_url + sub_url + '?page=' + str(1)
req = requests.get(url)
req.text

'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<!--[if lte IE 8]>\r\n<script>\r\n  location.replace(\'/support/explorer_upgrade.html\');\r\n</script>\r\n<![endif]-->\r\n<meta charset="utf-8"/>\r\n<title>한빛아카데미</title>\r\n<link rel="shortcut icon" href="https://www.hanbit.co.kr/images/common/hanbit.ico"> \r\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\r\n<meta property="og:type" content="website"/>\r\n<meta property="og:title" content="한빛아카데미"/>\r\n<meta property="og:description" content="이공계 분야 종합 교재 출판사로, 독자의 신뢰를 최우선 원칙으로 삼고 가르치는 교수님들과 배우는 학생들에게 든든한 조력자가 되도록 노력하고 있습니다."/>\r\n<meta property="og:image" content="https://www.hanbit.co.kr/images/hanbitpubnet_logo.jpg" />\r\n<meta property="og:url" content="https://www.hanbit.co.kr/academy/books/new_book_list.html?page=1"/>\r\n<link rel="canonical" href="https://www.hanbit.co.kr/academy/books/new_book_list.html?page=1" />\r\n<meta name="keywords" content="한빛아카데미,책,전자책,ebook,대학교재,교재출판사,강의교안,교수회원,자격증,정보교과서" />\r\n<meta name="de

In [7]:
soup = BeautifulSoup(req.text, 'html.parser') # 문법에 맞게 파싱을 하도록 하는 파라미터 parser. 만약 xml을 쓸거면 xml.parser처럼 쓰는듯

### 1. 찾으려고 하는 데이터

In [8]:
lis = soup.select('li.sub_book_list')
len(lis)

20

### 2. 하나의 데이터로 원하는 정보 추출

In [10]:
li = lis[0]
href = li.find('a')['href']
href

'/academy/books/book_view.html?p_code=B8672666879'

#### sub page로 이동해서 원하는 정보 추출

In [11]:
sub_req = requests.get(base_url + href)
sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

In [12]:
title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
title

'STEM CookBook, 문제 해결력을 키우는 기계설계'

In [19]:
# 번역자가 있는 책이 있고 없는 책이 있다.
# 딕셔너리로 각 항목을 저장하고 역자를 불러왔을 때 에러가 나오는 것을 예상해서 예외처리로 해결?
info_lis = sub_soup.select('.info_list > li')
info_dict = {}

for li in info_lis:
    key = li.find('strong').get_text().split(' ')[0]
    val = li.find('span').get_text().strip()
    info_dict[key] = val

info_dict

{'ISBN': '9791156646150',
 '물류코드': '4615',
 '저자': '이상범',
 '출간': '2022-08-05',
 '페이지': '680 쪽'}

In [21]:
author = info_dict['저자']
try:
    translator = info_dict['번역']
except:
    translator = '-'
date = info_dict['출간']
page = int(info_dict['페이지'].split()[0].replace(',',''))
author, translator, date, page

('이상범', '-', '2022-08-05', 680)

In [25]:
# 가격
price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',',''))
price

34000

### 정보 총 정리

In [26]:
data = []
for i in range(1, 11):
    base_url = 'https://www.hanbit.co.kr'
    sub_url = '/academy/books/new_book_list.html'
    url = base_url + sub_url + '?page=' + str(i)
    req = requests.get(url)

    lis = soup.select('li.sub_book_list')
    for li in lis:
        href = li.find('a')['href']
        sub_req = requests.get(base_url + href)
        sub_soup = BeautifulSoup(sub_req.text, 'html.parser')
        title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
        info_lis = sub_soup.select('.info_list > li')
        info_dict = {}

        for l in info_lis:
            key = l.find('strong').get_text().split(' ')[0]
            val = l.find('span').get_text().strip()
            info_dict[key] = val

        author = info_dict['저자']
        try:
            translator = info_dict['번역']
        except:
            translator = '-'
        date = info_dict['출간']
        page = int(info_dict['페이지'].split()[0].replace(',',''))
        price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',',''))

        data.append([title, author, translator, date, page, price])

In [ ]:
df = pd.DataFrame(data, columns=['제목','저자','번역','출간일','페이지수','판매가'])
df

In [28]:
#tqdm 써보기
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
data = []
for i in tqdm(range(1, 11)):
    url = base_url + sub_url + '?page=' + str(i)
    req = requests.get(url)
    lis = soup.select('li.sub_book_list')

    for li in lis:
        href = li.find('a')['href']
        sub_req = requests.get(base_url + href)
        sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

        title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
        info_lis = sub_soup.select('.info_list > li')
        
        info_dict = {}
        for l in info_lis:
            key = l.find('strong').get_text().split(' ')[0]
            val = l.find('span').get_text().strip()
            info_dict[key] = val

        author = info_dict['저자']
        try:
            translator = info_dict['번역']
        except:
            translator = '-'
        date = info_dict['출간']
        page = int(info_dict['페이지'].split()[0].replace(',',''))
        price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',',''))

        data.append([title, author, translator, date, page, price])

100%|██████████| 10/10 [02:13<00:00, 13.38s/it]


In [29]:
df = pd.DataFrame(data, columns=['제목','저자','번역','출간일','페이지수','판매가'])
df

,제목,저자,번역,출간일,페이지수,판매가
0,"STEM CookBook, 문제 해결력을 키우는 기계설계",이상범,-,2022-08-05,680,34000
1,경영분석(6판),"장영광 , 송치승",-,2022-07-22,556,36000
2,"IT CookBook, 기초부터 시작하는 PLC : OMRON NX","정완보 , 이진호",-,2022-07-18,636,32000
3,"IT CookBook, 원리로 이해하는 전력전자공학(2판)","원충연 , 김영렬 , 이택기 , 류홍제 , 故정용채",-,2022-07-11,588,30000
4,"IT CookBook, TCP/IP 소켓 프로그래밍(2판)",김선우,-,2022-06-30,600,32000
...,...,...,...,...,...,...
195,"IT CookBook, UI/UX 디자인 이론과 실무 with 어도비 XD",이영주,-,2022-05-20,512,30000
196,"STEM@CookBook, 공학 기초수학(2판)",함남우,-,2022-05-02,484,22000
197,"STEM@CookBook, Beer의 정역학과 재료역학(3판)","Ferdinand P. Beer , E. Russell Johnston, Jr. ,...","김경식 , 박재균 , 오창국 , 이정휘",2022-03-22,772,39000
198,STEIN 푸리에 해석학,"ELIAS M. STEIN , RAMI SHAKARCHI","한빛수학교재연구소 , 이상혁(감수)",2022-03-18,348,38000


In [31]:
import numpy as np
len(np.unique(df['제목']))
#어디선가 잘못됨..

20

In [33]:
#tqdm 써보기
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
data = []
for i in tqdm(range(1, 11)):
    url = base_url + sub_url + '?page=' + str(i)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser') #soup을 새로 만들지 않았음.
    lis = soup.select('li.sub_book_list')

    for li in lis:
        href = li.find('a')['href']
        sub_req = requests.get(base_url + href)
        sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

        title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
        info_lis = sub_soup.select('.info_list > li')
        
        info_dict = {}
        for l in info_lis:
            key = l.find('strong').get_text().split(' ')[0]
            val = l.find('span').get_text().strip()
            info_dict[key] = val

        author = info_dict['저자']
        try:
            translator = info_dict['번역']
        except:
            translator = '-'
        date = info_dict['출간']
        page = int(info_dict['페이지'].split()[0].replace(',',''))
        price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',',''))

        data.append([title, author, translator, date, page, price])

100%|██████████| 10/10 [01:50<00:00, 11.01s/it]


In [34]:
df = pd.DataFrame(data, columns=['제목','저자','번역','출간일','페이지수','판매가'])
df

,제목,저자,번역,출간일,페이지수,판매가
0,"STEM CookBook, 문제 해결력을 키우는 기계설계",이상범,-,2022-08-05,680,34000
1,경영분석(6판),"장영광 , 송치승",-,2022-07-22,556,36000
2,"IT CookBook, 기초부터 시작하는 PLC : OMRON NX","정완보 , 이진호",-,2022-07-18,636,32000
3,"IT CookBook, 원리로 이해하는 전력전자공학(2판)","원충연 , 김영렬 , 이택기 , 류홍제 , 故정용채",-,2022-07-11,588,30000
4,"IT CookBook, TCP/IP 소켓 프로그래밍(2판)",김선우,-,2022-06-30,600,32000
...,...,...,...,...,...,...
195,"IT Cookbook, 처음 만나는 전자회로",황형수,-,2019-07-10,440,24000
196,스트랭 미분방정식과 선형대수학,Gilbert Strang,한빛수학교재연구소,2019-07-09,552,30000
197,"IT CookBook, 네트워크 개론(3판)",진혜진,-,2019-07-05,516,25000
198,"IT CookBook, 텐서플로로 배우는 딥러닝","박혜정 , 석경하 , 심주용 , 황창하",-,2019-06-30,448,27000


In [38]:
len(np.unique(df['제목']))

199

In [39]:
df.to_csv('한빛신간도서(220808).csv', index= False)